# Demo 1
## Connecting to mongodb using python

In [1]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')

In [2]:
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.thedatasocietydb
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'thedatasocietydb')

## Insert

* Não há regras de validação rígidas: qualquer tipo de documento pode ser armazenado no banco de dados. 

* Na prática, somente objetos semelhantes são armazenados na base de dados, porém se um dia aparecer um novo atributo, poderemos inclui-lo apenas nos documentos aonde o mesmo é necessário, ao contrário do modelo relacional, onde uma nova coluna seria necessária e seria aplicada a todos os registros.

* Desvantagem: 

    * Cuidado para que as bases de dados não virem uma grande confusão (vão virar, aceite desde o início)
    
    
* Vantagens:

    * Temos uma abordagem n-dimensional para representar objetos (bem… o mundo muda)
    
    * Facilidade de alteração da estrutura de dados (refatoração fria)



In [3]:
# Insere um documento na coleção autores

db.autores.insert(
  {
    "idade": 25,
    "autor": "Joao"
  }
)

ObjectId('5d44e9d105c85aee1d563b41')

In [4]:
#Insere outro documento com estrutura diferente em autores

db.autores.insert(
  {
    "autor": "Joaquim",
    "idade": 30,
    "cidade": "Campinas"
  }
)

ObjectId('5d44e9d105c85aee1d563b42')

In [5]:
# Insere outro documento com estrutura diferente em autores

db.autores.insert(
  {
      "nome": "Maria",
    "poemaPreferido": "Desejo – Victor Hugo"
  }
)

ObjectId('5d44e9d105c85aee1d563b43')

In [6]:
# Insere documento com estrutura diferente que possui outro documento (publicacao) embutido (embedded)

db.autores.insert(
  {
    "autor": "Horacio",
    "publicacao": {    "tipo": "livro",
                "titulo": "Vida dos Dinossauros",
                "ano": 2010
               }
  }
)

ObjectId('5d44e9d105c85aee1d563b44')

## Count

In [7]:
# db.thedatasocietydb.count()

db.autores.count_documents({})

4

## Find

Structure:

```bash
db.collection.find({query},{projection});
```


In [8]:
# Lista todos os documentos da coleção autores

result = db.autores.find({})

for document in result: 
    pprint(document)
    print()

{'_id': ObjectId('5d44e9d105c85aee1d563b41'), 'autor': 'Joao', 'idade': 25}

{
    '_id': ObjectId('5d44e9d105c85aee1d563b42'),
    'autor': 'Joaquim',
    'cidade': 'Campinas',
    'idade': 30,
}

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'nome': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
}

{
    '_id': ObjectId('5d44e9d105c85aee1d563b44'),
    'autor': 'Horacio',
    'publicacao': {
        'ano': 2010,
        'tipo': 'livro',
        'titulo': 'Vida dos Dinossauros',
    },
}



In [9]:
# Lista os documentos de autores com nome = Maria

result = db.autores.find({"nome": "Maria"})

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'nome': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
}


In [10]:
# Lista poemaPreferido de documentos de autores com nome = Maria

result = db.autores.find(
                         {"nome": "Maria"}, 
                         {
                          "_id": 0, 
                          "poemaPreferido": 1
                         }
                    )

for document in result: 
    pprint(document)

{'poemaPreferido': 'Desejo – Victor Hugo'}


In [11]:
# Lista documentos de autores com tipo da publicacao = livro

result = db.autores.find({"publicacao.tipo": "livro"})

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b44'),
    'autor': 'Horacio',
    'publicacao': {
        'ano': 2010,
        'tipo': 'livro',
        'titulo': 'Vida dos Dinossauros',
    },
}


In [12]:
# Documentos de autores com tipo = livro e ano entre 2000 e 2015

result = db.autores.find(
                          {
                            "publicacao.tipo": "livro", 
                            "publicacao.ano": {"$gt": 1999, "$lt": 2016}
                          }
                        )
for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b44'),
    'autor': 'Horacio',
    'publicacao': {
        'ano': 2010,
        'tipo': 'livro',
        'titulo': 'Vida dos Dinossauros',
    },
}


## Update

In [13]:
# Atualiza cidade de autor = Joaquim

db.autores.update(
                   { "autor" : "Joaquim" },
                   { "$set": { 
                               "cidade": "Campinas-SP" 
                             } 
                   }
)


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [14]:
result = db.autores.find({ "autor" : "Joaquim" })

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b42'),
    'autor': 'Joaquim',
    'cidade': 'Campinas-SP',
    'idade': 30,
}


In [15]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'nome': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
}


In [16]:
# Adiciona atributo cidade ao autor de nome = Maria

db.autores.update(
    {"nome": "Maria"}, 
    {"$set": {"cidade": "Campinas"}}
)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [17]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'cidade': 'Campinas',
    'nome': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
}


In [18]:
# Exclui atributo poemaPreferido ao autor de nome = Maria
db.autores.update(
    {"nome": "Maria"}, 
    {"$unset": {"poemaPreferido": 1}}
)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [19]:
result = db.autores.find({ "nome" : "Maria" })

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'cidade': 'Campinas',
    'nome': 'Maria',
}


In [20]:
# Sobreescreve documento inteiro cujo nome = Maria
db.autores.update(
  { "nome":  "Maria" },
  { "autor": "Maria",
    "poemaPreferido": "Desejo – Victor Hugo",
    "publicacao": { 
                    "tipo": "livro",
                    "titulo": "Antes da Extinção",
                    "ano": 2014
              }
  }
)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [23]:
result = db.autores.find({ "autor" : "Maria" })

for document in result: 
    pprint(document)

{
    '_id': ObjectId('5d44e9d105c85aee1d563b43'),
    'autor': 'Maria',
    'poemaPreferido': 'Desejo – Victor Hugo',
    'publicacao': {
        'ano': 2014,
        'tipo': 'livro',
        'titulo': 'Antes da Extinção',
    },
}


### Operadores Update

* **upsert**: Opcional. Se = true, cria um novo documento quando não encontra um que atende ao critério de busca. Default = false.

* **multi:** Opcional. Se = true, atualiza todos os documentos que atendem ao critério de busca. Se = false, atualiza apenas o primeiro documento. Default = false.

In [25]:
# Atualiza todos os documentos com titulo Vida dos Dinossauros
db.autores.update(
                  { "publicacao.titulo":"Vida dos Dinossauros"},
                  { "$set": {"publicacao.ano": 2011}},
                  False, #upsert
                  True   #multi
)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

## Remove

In [27]:
# Exclui documento cujo autor = Maria

db.autores.remove({"autor": "Maria"})

{'n': 0, 'ok': 1.0}

In [29]:
# Exclui todos os documentos da coleção

db.autores.remove({})

{'n': 0, 'ok': 1.0}

## Referencing